In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 30.1 MB/s eta 0:00:00


In [2]:
import torch
from ultralytics import YOLO
import os


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model_path = "/kaggle/input/best-model-roboflow/best.pt"  # Update with correct path
model = YOLO(model_path)


In [4]:
dataset_yaml = """
path: /kaggle/input/chest-xray-pneumonia/chest_xray  # Base dataset path
train: train  # Train images folder
val: val  # Validation images folder
test: test  # Test images folder

nc: 2  # Number of classes (Normal, Pneumonia)
names: ['Normal', 'Pneumonia']
"""

with open("dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("✅ dataset.yaml created successfully!")


✅ dataset.yaml created successfully!


In [7]:
from ultralytics import YOLO
import os
import torch
import gc
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Load YOLO Model (Use GPU if available, else CPU)
model_path = "/kaggle/input/best-model-roboflow/best.pt"  
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(model_path).to(device)

# Define Dataset Paths
dataset_path = "/kaggle/input/chest-xray-pneumonia/chest_xray"
splits = ["train", "val", "test"]
categories = ["NORMAL", "PNEUMONIA"]

# ✅ Initialize dictionary to store evaluation results
results = {}

def evaluate_model(split):
    y_true = []
    y_pred = []
    total_images = 0  # Counter to track processed images

    print(f"\nProcessing {split} dataset...")

    for category in categories:
        folder_path = os.path.join(dataset_path, split, category)

        for idx, image_name in enumerate(os.listdir(folder_path)):
            image_path = os.path.join(folder_path, image_name)

            # ✅ FIX: Use a different variable name instead of 'results'
            yolo_results = model(image_path, verbose=False)  # Avoid overwriting 'results' dict

            # Extract Detected Class Labels
            detected_classes = []
            for result in yolo_results:
                if result.boxes is not None:
                    detected_classes.extend(result.boxes.cls.tolist())

            # Determine Final Prediction
            pred_class = max(set(detected_classes), key=detected_classes.count) if detected_classes else 0
            actual_class = 0 if category == "NORMAL" else 1

            # Store Ground Truth & Predictions
            y_true.append(actual_class)
            y_pred.append(pred_class)

            # Update Image Counter
            total_images += 1
            print(f"Processed Image {total_images}: {image_path}")

            # Free GPU Memory Every 50 Iterations
            if idx % 50 == 0:
                torch.cuda.empty_cache()
                gc.collect()

    # Compute Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Compute Evaluation Metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    specificity = (tn / (tn + fp)) * 100
    f1 = f1_score(y_true, y_pred) * 100
    fpr = (fp / (fp + tn)) * 100

    # ✅ FIX: Store results correctly
    results[split] = {
        "Accuracy (%)": round(accuracy, 2),
        "Precision (%)": round(precision, 2),
        "Recall (Sensitivity) (%)": round(recall, 2),
        "Specificity (%)": round(specificity, 2),
        "F1 Score (%)": round(f1, 2),
        "False Positive Rate (FPR) (%)": round(fpr, 2),
    }

    print(f"\nFinished processing {split} dataset. Total images processed: {total_images}")

# Run Evaluation on all splits
for split in splits:
    evaluate_model(split)

# Convert results to DataFrame
df = pd.DataFrame.from_dict(results, orient="index")

# Print formatted table
print("\nFinal Evaluation Metrics:")
print(df.to_markdown(tablefmt="fancy_grid"))



Processing train dataset...
Processed Image 1: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-0771-0001.jpeg
Processed Image 2: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-1294-0001-0002.jpeg
Processed Image 3: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/IM-0675-0001.jpeg
Processed Image 4: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-1169-0001.jpeg
Processed Image 5: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/IM-0421-0001.jpeg
Processed Image 6: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-0531-0001.jpeg
Processed Image 7: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-0416-0001-0002.jpeg
Processed Image 8: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-0965-0001.jpeg
Processed Image 9: /kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/NORMAL2-IM-0627-0001.jpeg
Processed Image 10: /kaggle/input/chest-x

In [11]:
import pandas as pd

# Rename columns for conciseness
df = df.rename(columns={
    "Accuracy (%)": "Acc (%)",
    "Precision (%)": "Prec (%)",
    "Recall (Sensitivity) (%)": "Rec (%)",
    "Specificity (%)": "Spec (%)",
    "F1 Score (%)": "F1 (%)",
    "False Positive Rate (FPR) (%)": "FPR (%)"
})

# Round metrics to 2 decimal places
df = df.round(2)

# Print neatly formatted table
print("\nFinal Evaluation Metrics for Journal:")
print(df.to_markdown(tablefmt="fancy_grid"))

# Save in multiple formats
df.to_csv("evaluation_metrics.csv", index=True)      # CSV format
df.to_latex("evaluation_metrics.tex", index=True)    # LaTeX format (for journals)



Final Evaluation Metrics for Journal:
╒═══════╤═══════════╤════════════╤═══════════╤════════════╤══════════╤═══════════╕
│       │   Acc (%) │   Prec (%) │   Rec (%) │   Spec (%) │   F1 (%) │   FPR (%) │
╞═══════╪═══════════╪════════════╪═══════════╪════════════╪══════════╪═══════════╡
│ train │     91.72 │      97.39 │     91.3  │      92.92 │    94.25 │      7.08 │
├───────┼───────────┼────────────┼───────────┼────────────┼──────────┼───────────┤
│ val   │     62.5  │      60    │     75    │      50    │    66.67 │     50    │
├───────┼───────────┼────────────┼───────────┼────────────┼──────────┼───────────┤
│ test  │     82.21 │      81.07 │     93.33 │      63.68 │    86.77 │     36.32 │
╘═══════╧═══════════╧════════════╧═══════════╧════════════╧══════════╧═══════════╛


In [18]:
import pandas as pd

# Manually set the total number of images per dataset split
image_counts = {"train": 5216, "val": 16, "test": 624}

# If "Images" column already exists, update it; otherwise, insert it
if "Images" in df.columns:
    df["Images"] = df.index.map(image_counts)
else:
    df.insert(2, "Images", df.index.map(image_counts))

# Rename columns for conciseness
df = df.rename(columns={
    "Accuracy (%)": "Acc (%)",
    "Precision (%)": "Prec (%)",
    "Recall (Sensitivity) (%)": "Rec (%)",
    "Specificity (%)": "Spec (%)",
    "F1 Score (%)": "F1 (%)",
    "False Positive Rate (FPR) (%)": "FPR (%)"
})

# Round metrics to 2 decimal places
df = df.round(2)

# Select train and test for the final formatted table
df_combined = df.loc[["train", "test"]]

# Print the structured table with train & test metrics
print("\nFinal Evaluation Metrics (Train & Test):")
print(df_combined.to_markdown(tablefmt="fancy_grid"))

# Save in multiple formats for journal submission
df_combined.to_csv("evaluation_metrics_train_test.csv", index=True)      # CSV format
df_combined.to_latex("evaluation_metrics_train_test.tex", index=True)    # LaTeX format (for journals)



Final Evaluation Metrics (Train & Test):
╒═══════╤═══════════╤════════════╤═══════════╤════════════╤══════════╤═══════════╤══════════╕
│       │   Acc (%) │   Prec (%) │   Rec (%) │   Spec (%) │   F1 (%) │   FPR (%) │   Images │
╞═══════╪═══════════╪════════════╪═══════════╪════════════╪══════════╪═══════════╪══════════╡
│ train │     91.72 │      97.39 │     91.3  │      92.92 │    94.25 │      7.08 │     5216 │
├───────┼───────────┼────────────┼───────────┼────────────┼──────────┼───────────┼──────────┤
│ test  │     82.21 │      81.07 │     93.33 │      63.68 │    86.77 │     36.32 │      624 │
╘═══════╧═══════════╧════════════╧═══════════╧════════════╧══════════╧═══════════╧══════════╛


In [23]:
print(df.columns)
print(df)


Index(['Acc (%)', 'Prec (%)', 'Rec (%)', 'Spec (%)', 'F1 (%)', 'FPR (%)',
       'Images'],
      dtype='object')
       Acc (%)  Prec (%)  Rec (%)  Spec (%)  F1 (%)  FPR (%)  Images
train    91.72     97.39    91.30     92.92   94.25     7.08  5216.0
val      62.50     60.00    75.00     50.00   66.67    50.00     NaN
test     82.21     81.07    93.33     63.68   86.77    36.32   624.0

Final Combined Metrics (Train + Test):
╒══════════════╤══════════╤═══════════╤════════════╤═══════════╤════════════╤══════════╤═══════════╕
│              │   Images │   Acc (%) │   Prec (%) │   Rec (%) │   Spec (%) │   F1 (%) │   FPR (%) │
╞══════════════╪══════════╪═══════════╪════════════╪═══════════╪════════════╪══════════╪═══════════╡
│ Train + Test │     5840 │      90.7 │      95.65 │     91.52 │       89.8 │    93.45 │      10.2 │
╘══════════════╧══════════╧═══════════╧════════════╧═══════════╧════════════╧══════════╧═══════════╛


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [27]:
import pandas as pd

# Define total images for train, val, and test
image_counts = {"train": 5216, "val": 12, "test": 624}

# Ensure "Images" column is correctly assigned
for split in image_counts:
    df.loc[split, "Images"] = image_counts[split]

# Select all three splits (train, val, test)
df_all = df.loc[["train", "val", "test"]]

# Compute total images across all splits
total_images = df_all["Images"].sum()

# Compute weighted averages for each metric
final_metrics = {"Images": total_images}

metrics_cols = [col for col in df.columns if col != "Images"]

for col in metrics_cols:
    final_metrics[col] = sum(df_all.loc[split, col] * image_counts[split] for split in image_counts) / total_images

# Convert to DataFrame
df_final = pd.DataFrame(final_metrics, index=["Train + Val + Test"]).round(2)

# Print final table
print("\nFinal Combined Metrics for Kaggle Chest X-ray Dataset:")
print(df_final.to_markdown(tablefmt="fancy_grid"))

# Save in multiple formats
df_final.to_csv("final_combined_metrics.csv", index=True)
df_final.to_latex("final_combined_metrics.tex", index=True)



Final Combined Metrics for Kaggle Chest X-ray Dataset:
╒════════════════════╤══════════╤═══════════╤════════════╤═══════════╤════════════╤══════════╤═══════════╕
│                    │   Images │   Acc (%) │   Prec (%) │   Rec (%) │   Spec (%) │   F1 (%) │   FPR (%) │
╞════════════════════╪══════════╪═══════════╪════════════╪═══════════╪════════════╪══════════╪═══════════╡
│ Train + Val + Test │     5852 │     90.65 │      95.57 │     91.48 │      89.71 │     93.4 │     10.29 │
╘════════════════════╧══════════╧═══════════╧════════════╧═══════════╧════════════╧══════════╧═══════════╛


In [30]:
import pandas as pd

# Define total images for train, val, and test
image_counts = {"train": 5216, "val": 12, "test": 624}

# Ensure "Images" column is correctly assigned
for split in image_counts:
    df.loc[split, "Images"] = image_counts[split]

# Select all three splits (train, val, test)
df_all = df.loc[["train", "val", "test"]]

# Compute total images across all splits
total_images = df_all["Images"].sum()

# Define full metric names
metric_names = {
    "Acc (%)": "Accuracy (%)",
    "Prec (%)": "Precision (%)",
    "Rec (%)": "Recall (Sensitivity, TPR) (%)",
    "Spec (%)": "Specificity (TNR) (%)",
    "F1 (%)": "F1 Score (%)",
    "FPR (%)": "False Positive Rate (FPR) (%)",
    "Images": "Total Images Processed"
}

# Compute weighted averages for each metric
final_metrics = {"Total Images Processed": total_images}

metrics_cols = [col for col in df.columns if col != "Images"]

for col in metrics_cols:
    final_metrics[metric_names[col]] = sum(df_all.loc[split, col] * image_counts[split] for split in image_counts) / total_images

# Convert to DataFrame with full metric names
df_final = pd.DataFrame(final_metrics.items(), columns=["Metric", "Value"]).round(4)

# Print final table
print("\nFinal Combined Metrics for Kaggle Chest X-ray Dataset:")
print(df_final.to_markdown(tablefmt="fancy_grid"))

# Save in multiple formats
df_final.to_csv("final_combined_metrics.csv", index=False)
df_final.to_latex("final_combined_metrics.tex", index=False)



Final Combined Metrics for Kaggle Chest X-ray Dataset:
╒════╤═══════════════════════════════╤═══════════╕
│    │ Metric                        │     Value │
╞════╪═══════════════════════════════╪═══════════╡
│  0 │ Total Images Processed        │ 5852      │
├────┼───────────────────────────────┼───────────┤
│  1 │ Accuracy (%)                  │   90.646  │
├────┼───────────────────────────────┼───────────┤
│  2 │ Precision (%)                 │   95.5731 │
├────┼───────────────────────────────┼───────────┤
│  3 │ Recall (Sensitivity, TPR) (%) │   91.483  │
├────┼───────────────────────────────┼───────────┤
│  4 │ Specificity (TNR) (%)         │   89.7141 │
├────┼───────────────────────────────┼───────────┤
│  5 │ F1 Score (%)                  │   93.3959 │
├────┼───────────────────────────────┼───────────┤
│  6 │ False Positive Rate (FPR) (%) │   10.2859 │
╘════╧═══════════════════════════════╧═══════════╛
